In [116]:
import warnings
warnings.filterwarnings('ignore')

### Modelos preentrenados de Keras

En un principio se probarán 3 modelos.

    - VGG-16
    - Inception V3
    - ResNet 50

In [125]:
from keras.applications import VGG16
from keras.applications import InceptionV3
from keras.applications import ResNet50V2

from keras.applications.vgg16 import preprocess_input as vgg16_preprocessor
from keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor
from keras.applications.resnet_v2 import preprocess_input as resnet_v2_preprocessor


from keras.models import clone_model
from keras.models import Model
from keras.preprocessing import image


from IPython.display import clear_output
from tqdm import tqdm


models_dict = {}
vgg16_dict, inception_v3_dict, resnet50_dict = {} ,{} ,{}


print("Loading VGG16")
model = VGG16(weights='imagenet')
model = Model(model.input, model.layers[-2].output) # output_shape de la penúltima capa(fully-connected) -> (,4096)
vgg16_dict["model"] = clone_model(model)
vgg16_dict["preprocesor"] = vgg16_preprocessor
vgg16_dict["target_size"] = model.input_shape[1], model.input_shape[2]# default vgg16 input (224,224)


print("Loading Inception v3")
model = InceptionV3(weights='imagenet')
model = Model(model.input, model.layers[-2].output)# output_shape de la penúltima capa(pooling) -> (,2048)
inception_v3_dict["model"] = clone_model(model)
inception_v3_dict["preprocesor"] = inception_v3_preprocessor
inception_v3_dict["target_size"] = model.input_shape[1],model.input_shape[2] # default inceptionv3 input (299,299)


print("Loading ResNet 50")
model = ResNet50V2(weights='imagenet')
model = Model(model.input, model.layers[-2].output)# output_shape de la penúltima capa(pooling) -> (,2048)
resnet50_dict["model"] = clone_model(model)
resnet50_dict["preprocesor"] = resnet_v2_preprocessor
resnet50_dict["target_size"] = model.input_shape[1],model.input_shape[2]# default ResNet input (224,224)


# diccionarios de los 3 modelos, clave -> nombre, valor -> otro diccionario
models_dict["VGG16"] = vgg16_dict
models_dict["InceptionV3"] = inception_v3_dict
models_dict["ResNet50"] = resnet50_dict

clear_output()

In [126]:
import os
import cv2
import numpy as np
from keras.preprocessing import image
from skimage.io import imsave
import uuid

import matplotlib.pyplot as plt

def extract_features(image_path,model_name):
    '''
    model_name extaerá los features de la imagen que se le pase como parámetro.
    '''
    model_dict = models_dict[model_name]
    ####
    model = model_dict["model"]
    preprocessor = model_dict["preprocesor"]
    target_size = model_dict["target_size"]
    # se carga la imagen y después se ajusta al input shape del modelo
    img = cv2.resize(cv2.imread(image_path),target_size)

    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocessor(img_data)
    
    # extracción de features
    features = model.predict(img_data)
    return features[0]

In [127]:
def get_paths(directory):
    '''
    Devuelve la ruta relativa de las muestras .bmp
    '''
    paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".bmp"):
                paths.append(os.path.join(root, file))
    return paths

In [128]:
# paths de las muestra polarizadas
images = get_paths("CASIA-Polar")

In [129]:
for model_name in models_dict:
    print(model_name)

VGG16
InceptionV3
ResNet50


### Extracción de features de una sola muestra

Demo para comprobar que los 3 modelos extraen los features correctamente.

In [130]:
# features con modelo VGG16
feat_16 = extract_features(images[0],"VGG16")
print(feat_16.shape, feat_16[:10])

(4096,) [0.15759276 0.48433328 0.         0.1136077  0.         0.35416684
 0.15575247 0.         0.01177126 0.        ]


In [131]:
# features con modelo InceptionV3
feat_inceptionV3 = extract_features(images[0],"InceptionV3")
print(feat_inceptionV3.shape, feat_inceptionV3[:10])

(2048,) [1.59658189e-03 1.17165524e-04 0.00000000e+00 1.61122327e-04
 5.58187312e-04 7.85618977e-06 4.39360330e-04 1.60738218e-05
 1.14461861e-03 1.07891613e-03]


In [132]:
# features con modelo ResNet
feat_r50 = extract_features(images[0],"ResNet50")
print(feat_r50.shape, feat_r50[:10])

(2048,) [5.9741527e-02 1.2129251e-02 2.6713071e-02 2.1642070e-02 5.6126883e-04
 7.0216302e-03 9.1353506e-03 0.0000000e+00 1.1688420e-03 3.0627412e-05]


In [133]:
import pandas as pd

# cargamos csv con los datos de las muestras
df = pd.read_csv("iris_data.csv", index_col=0) # quitamos columna unnamed

# cambiamos nombre de columnas para evitar conflictos con palabras reservadas de Pytho
df.rename(columns={'image':'Image','class':'Clase'}, inplace=True)
df.head()

,Image,pupil x_center,pupil y_center,pupil radius,iris x_center,iris y_center,iris radius,Clase
0,Adrianna_1.bmp,182,134,36,177,134,102,Adrianna
1,Adrianna_2.bmp,174,138,38,172,138,38,Adrianna
2,Adrianna_3.bmp,173,120,38,167,120,104,Adrianna
3,Adrianna_4.bmp,183,122,40,185,122,38,Adrianna
4,Adrianna_5.bmp,177,145,38,179,143,36,Adrianna


In [138]:
def register_to_deepfeatures(register):
    """
    Compute all CNN features    
    
    Parameters
    ----------
    register : Series
        Serie containing metadata of the image
   
    Returns
    -------
    results : Series 
        A Pandas Serie contanining all of the features

    """
    img_name = register.Image # nombre de la muestra e.g: Rafael_1.bmp
    clase = register.Clase # clase e.g Rafael
    
    img_dir = os.path.join("CASIA-Polar",clase)
    img_path = os.path.join(img_dir,img_name) # ruta completa de la muestra

    basic_values = pd.Series([img_name,clase],["Name","Class"])

    all_features = []
    all_names = []
    models = models_dict.keys() # 3 modelos
    
    for model in models:
        features = extract_features(img_path,model)
        names = [f"{model}_{i}" for i in range(len(features))]
        all_features+=list(features)
        all_names+= names
        
    results = pd.Series(all_features,all_names)
    return pd.concat((basic_values,results))


In [139]:
# probamos con una fila del dataframe
example1 = df.iloc[5]
example1

Image             Adrianna_6.bmp
pupil x_center               178
pupil y_center               134
pupil radius                  38
iris x_center                179
iris y_center                132
iris radius                   40
Clase                   Adrianna
Name: 5, dtype: object

In [140]:
results = register_to_deepfeatures(example1)
results

Name             Adrianna_6.bmp
Class                  Adrianna
VGG16_0                0.163629
VGG16_1                0.462037
VGG16_2                       0
VGG16_3                 0.06538
VGG16_4                       0
VGG16_5                 0.39014
VGG16_6                0.168639
VGG16_7                       0
VGG16_8               0.0201233
VGG16_9                       0
VGG16_10                0.15908
VGG16_11                      0
VGG16_12             0.00708534
VGG16_13             0.00761905
VGG16_14                      0
VGG16_15              0.0346056
VGG16_16                      0
VGG16_17                0.34648
VGG16_18                      0
VGG16_19                      0
VGG16_20                      0
VGG16_21                      0
VGG16_22              0.0629325
VGG16_23                      0
VGG16_24                0.34622
VGG16_25              0.0178438
VGG16_26               0.164041
VGG16_27                      0
                      ...      
ResNet50

In [141]:
# aplica la función a todas las FILAS del dataframe(parecido a map())
df_deep = df.apply(register_to_deepfeatures, axis=1)

In [153]:
import winsound
freq = 1500
dur = 1000

In [143]:
df_deep

,Name,Class,VGG16_0,VGG16_1,VGG16_2,VGG16_3,VGG16_4,VGG16_5,VGG16_6,VGG16_7,...,ResNet50_2038,ResNet50_2039,ResNet50_2040,ResNet50_2041,ResNet50_2042,ResNet50_2043,ResNet50_2044,ResNet50_2045,ResNet50_2046,ResNet50_2047
0,Adrianna_1.bmp,Adrianna,0.157593,0.484333,0.000000,0.113608,0.000000,0.354167,0.155752,0.000000,...,0.000000,0.051367,0.054490,0.004548,0.000903,0.025169,0.003536,0.059130,0.028055,0.085800
1,Adrianna_2.bmp,Adrianna,0.180519,0.471455,0.000000,0.092951,0.000000,0.342986,0.135956,0.000000,...,0.000062,0.050364,0.051624,0.003743,0.001588,0.026318,0.002470,0.059326,0.026802,0.082919
2,Adrianna_3.bmp,Adrianna,0.104904,0.452698,0.000000,0.087041,0.000000,0.322919,0.140523,0.000000,...,0.000000,0.048856,0.054037,0.003512,0.000691,0.023847,0.002301,0.058310,0.024932,0.086808
3,Adrianna_4.bmp,Adrianna,0.159594,0.422320,0.000000,0.100514,0.000000,0.352374,0.166457,0.000000,...,0.000000,0.051447,0.052351,0.003461,0.001101,0.024022,0.002699,0.057001,0.024433,0.084384
4,Adrianna_5.bmp,Adrianna,0.154488,0.491368,0.000000,0.094237,0.000000,0.374492,0.171313,0.000000,...,0.000000,0.051142,0.058145,0.005316,0.001235,0.024091,0.003454,0.062244,0.028011,0.085642
5,Adrianna_6.bmp,Adrianna,0.163629,0.462037,0.000000,0.065380,0.000000,0.390140,0.168639,0.000000,...,0.000026,0.051961,0.057726,0.004894,0.000688,0.024678,0.002067,0.060765,0.028243,0.085984
6,Adrianna_7.bmp,Adrianna,0.162765,0.494493,0.000427,0.051270,0.015384,0.355746,0.174316,0.000000,...,0.000138,0.051409,0.055090,0.004040,0.000092,0.023309,0.003397,0.061438,0.026281,0.089180
7,Aharon_1.bmp,Aharon,0.152619,0.447010,0.000000,0.037086,0.000000,0.358575,0.138043,0.000000,...,0.000000,0.049593,0.052027,0.002665,0.000354,0.024648,0.003831,0.058081,0.021285,0.081568
8,Aharon_2.bmp,Aharon,0.130353,0.482663,0.000000,0.018008,0.000000,0.365793,0.179917,0.000000,...,0.000000,0.050326,0.049775,0.004654,0.000228,0.024457,0.005334,0.057649,0.021241,0.083850
9,Aharon_3.bmp,Aharon,0.174064,0.499380,0.000000,0.013127,0.000000,0.403140,0.122394,0.000000,...,0.000216,0.052404,0.053088,0.004069,0.000322,0.022020,0.003259,0.057908,0.022413,0.083278


In [154]:
# guardamos las features
df_deep.to_csv("iris_deep_features.csv")

-----------------------------------

-------------------------

In [155]:
import pandas as pd
data = pd.read_csv("iris_deep_features.csv",index_col=0)
data.head()


,Name,Class,VGG16_0,VGG16_1,VGG16_2,VGG16_3,VGG16_4,VGG16_5,VGG16_6,VGG16_7,...,ResNet50_2038,ResNet50_2039,ResNet50_2040,ResNet50_2041,ResNet50_2042,ResNet50_2043,ResNet50_2044,ResNet50_2045,ResNet50_2046,ResNet50_2047
0,Adrianna_1.bmp,Adrianna,0.157593,0.484333,0.0,0.113608,0.0,0.354167,0.155752,0.0,...,0.000000,0.051367,0.054490,0.004548,0.000903,0.025169,0.003536,0.059130,0.028055,0.085800
1,Adrianna_2.bmp,Adrianna,0.180519,0.471455,0.0,0.092951,0.0,0.342986,0.135956,0.0,...,0.000062,0.050364,0.051624,0.003743,0.001588,0.026318,0.002470,0.059326,0.026802,0.082919
2,Adrianna_3.bmp,Adrianna,0.104904,0.452698,0.0,0.087041,0.0,0.322919,0.140523,0.0,...,0.000000,0.048856,0.054037,0.003512,0.000691,0.023847,0.002301,0.058310,0.024932,0.086808
3,Adrianna_4.bmp,Adrianna,0.159594,0.422320,0.0,0.100514,0.0,0.352374,0.166457,0.0,...,0.000000,0.051447,0.052351,0.003461,0.001101,0.024022,0.002699,0.057001,0.024433,0.084384
4,Adrianna_5.bmp,Adrianna,0.154488,0.491368,0.0,0.094237,0.0,0.374492,0.171313,0.0,...,0.000000,0.051142,0.058145,0.005316,0.001235,0.024091,0.003454,0.062244,0.028011,0.085642


In [156]:
data.shape

(756, 8194)

In [157]:
import pickle

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.pipeline import Pipeline

### Choose the right estimator

In [172]:
from sklearn.svm import LinearSVC # uno vs el resto
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# ensemble classifiers
from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import GradientBoostingClassifier

In [183]:
pretrained_models = ["VGG16","InceptionV3","ResNet50"]

In [184]:
# true labels
y = data.Class.values

In [185]:
vgg_cols = data.columns.str.startswith("VGG16")
inception_cols = data.columns.str.startswith("InceptionV3")
resnet_cols = data.columns.str.startswith("ResNet50")

In [186]:
vgg_X = data[data.columns[vgg_cols]].values
inception_X = data[data.columns[inception_cols]].values
resnet_X = data[data.columns[resnet_cols]].values

In [187]:
datasets = [(vgg_X,y), (inception_X,y), (resnet_X,y)]
datasets_names = pretrained_models.copy()

In [188]:
'''
Definition of the SVM parameter search
'''
import numpy as np
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid_svm = dict(gamma=gamma_range, C=C_range)
nested_cv = 5

grid_svm = GridSearchCV(SVC(), param_grid=param_grid_svm, cv=nested_cv)


In [189]:
'''
List of classifiers and their names included in the experimental study
'''

cls_names = ["SVM", "LogisticRegression", "Nearest Neighbors","Random Forest"] #"Gradient Boosting Trees"]

classifiers = [ make_pipeline(StandardScaler(), SVC(kernel='linear')), #(StandardScaler(), grid_svm)
                make_pipeline(StandardScaler(), KNeighborsClassifier(3)),
                make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000)),
                RandomForestClassifier(random_state=0, n_estimators=100),
#                 GradientBoostingClassifier(random_state=0, n_estimators=100)
                ]

### Validación cruzada para evaluar los clasificadores

In [192]:
def cross_validate_preds_model(X, y, model, num_folds):
        '''
        Perform cross validation with a model and a dataset (X and y),
        and returns the predictions to later obtain the measurements 
        you want
        
        Parameters
        ----------
        X: numpy.array
            Dataset (features)
        Y: numpy.array
            Dataset (Target)
        model: scikit_model
            model to be trained
        num_folds: int
            number of folds in the cross validation
        
        Return
        -------
        array 
            array of prediccions obtained using cross_validation
        '''
        print('\t'+str(model)[:20]+"...", end=' - ')
        preds = cross_val_predict(model,X,y,cv=num_folds)
        print('OK')
        
        return preds

def run_all_save(num_folds,filename):
    '''
    Perform cross validation with all models and datasets.
        
        
    Parameters
    ----------
    num_folds: int
        number of folds in the cross validation
    filename: string
        name of the file that stores the predictions obtained using crossvalidation
        
    Return
    -------
    
    ''' 
    
    all_preds = {}

    for dataset,dataset_name in tqdm(zip(datasets, dataset_names)):
        print(f"----------Pre-trained network: {dataset_name}-----------")
        X,y = dataset
        for model,cls_name in zip(classifiers,cls_names):
            print(f"-> {cls_name}")
            preds = cross_validate_preds_model(X, y, model, num_folds)
            all_preds[(dataset_name,cls_name)]=(y,preds)

    all_preds["cls_names"]=cls_names
    all_preds["dataset_names"]=dataset_names

    with open(filename, 'wb') as fp:
         pickle.dump(all_preds, fp)

In [193]:
obj_file = "deep_prueba.obj"

run_all_save(4,obj_file) # 4 folds (divisor de 756)
winsound.Beep(freq,dur)


0it [00:00, ?it/s]

----------Pre-trained network: VGG16-----------
-> SVM
	Pipeline(memory=None... - OK
-> LogisticRegression
	Pipeline(memory=None... - OK
-> Nearest Neighbors
	Pipeline(memory=None... - OK
-> Random Forest
	RandomForestClassifi... - OK



1it [07:25, 445.35s/it]

----------Pre-trained network: InceptionV3-----------
-> SVM
	Pipeline(memory=None... - OK
-> LogisticRegression
	Pipeline(memory=None... - OK
-> Nearest Neighbors
	Pipeline(memory=None... - OK
-> Random Forest
	RandomForestClassifi... - OK



2it [10:53, 374.14s/it]

----------Pre-trained network: ResNet50-----------
-> SVM
	Pipeline(memory=None... - OK
-> LogisticRegression
	Pipeline(memory=None... - OK
-> Nearest Neighbors
	Pipeline(memory=None... - OK
-> Random Forest
	RandomForestClassifi... - OK



3it [14:40, 329.99s/it]

In [194]:
import pickle

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [197]:
def conf_mat_df(cm,labels):
    '''
    Create a confusion matrix in a DataFrame
        
        
    Parameters
    ----------
    cm: ndarray 2D
        confusion matrix
    labels: list
        List of class names
        
    Return DataFrame
    -------
    
    ''' 

    return (pd.DataFrame(cm,index=labels, columns=labels)
          .rename_axis("actual")
          .rename_axis("predicted", axis=1))
    
#https://matplotlib.org/3.1.1/gallery/images_contours_and_fields/image_annotated_heatmap.html
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A list or array of two color specifications.  The first is used for
        values below a threshold, the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts


def get_results(filename):
    '''
    Load the file with the predictions.
    Compute accuracy, confusion matrix and other measures.
        
        
    Parameters
    ----------
    filename: string
        name of the file that stores the predictions obtained using crossvalidation
        
    Return
    dictionary
        A dictionary of key:values that asociates the name
        of a measure or chart with the value
    -------
    
    ''' 

    with open(filename, 'rb') as fp:
        all_preds = pickle.load(fp)

    cls_names = all_preds.pop("cls_names")
    dataset_names = all_preds.pop("dataset_names")

    data_cls_pairs = list(all_preds.keys())
    data_cls_pairs.sort()

    results = {}


    acc_df = pd.DataFrame(index=dataset_names, columns=cls_names)

    ## A DataFrame is created to store the accuracy in each clase
    for dataset in dataset_names:
        results[(dataset,"acc")] = pd.DataFrame(columns=cls_names)


    for dataset_name,cls_name in tqdm(data_cls_pairs):

        #print(dataset_name,cls_name)
        y_true, y_pred = all_preds[(dataset_name,cls_name)]
        labels = list(np.unique(y_true))

        acc = accuracy_score(y_true, y_pred)
        # Fill accuracy dataframe
        acc_df.at[dataset_name,cls_name]=acc

        # Get conf_mat
        cm = confusion_matrix(y_true, y_pred)
        cm_df = conf_mat_df(cm,labels)
        results[(dataset_name,cls_name,"cm")] = cm_df

        # Get classification report
        report = classification_report(y_true, y_pred, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        results[(dataset_name,cls_name,"report")] = report_df

        # Acc per class
        cm_dig = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_dig = cm_dig.diagonal()

        dfi = results[(dataset_name,"acc")]
        dfi[cls_name]=pd.Series(cm_dig,labels)    
        results[(dataset_name,"acc")]=dfi.copy()


    results["Acc"] = acc_df
    return results

In [198]:
results = get_results(obj_file)


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 47.18it/s]

In [199]:
results["Acc"]

,SVM,LogisticRegression,Nearest Neighbors,Random Forest
VGG16,0.886243,0.752646,0.899471,0.756614
InceptionV3,0.903439,0.777778,0.945767,0.813492
ResNet50,0.82672,0.687831,0.882275,0.638889
